In [1]:
import pandas as pd 
import numpy as np 



In [2]:
from utils.getinfo import IR_from_repayment_schedule
from utils.utils import *
from datetime import timedelta

def create_repayment_schedule_with_crrr(dataframe):
    rows = []
    previous_row_was_crr = False
    crr_info = {}
    days_in_month = 30
    
    # Retrieve monthly interest rate from the repayment schedule
    monthly_IR, _ = IR_from_repayment_schedule(dataframe)

    for index, row in dataframe.iterrows():
        opening_balance = rows[-1]['Closing Balance'] if rows else 0

        if row['Transaction Type'] == 'Capital Repayment Receivable':
            previous_row_was_crr = True
            crr_info = {
                'date': row['Installment Date'],
                'amount_paid': row['Principal Amount'],
                'closing_balance_before_crr': opening_balance,
                'closing_balance_after_crr': calculate_closing_balance(opening_balance, row['Principal Amount']),
                'days_before_crr': (row['Installment Date'] - dataframe.iloc[index - 1]['Installment Date']).days if index > 0 else 0
            }
            closing_balance = crr_info['closing_balance_after_crr']
            rows.append({
                'Installment Date': row['Installment Date'],
                'Transaction Type': row['Transaction Type'],
                'Opening Balance': round(opening_balance, 0),
                'Principal Amount': row['Principal Amount'],
                'Interest Amount': 0,
                'Installment Amount': 0,
                'Closing Balance': closing_balance
            })
        else:
            principal_amount, interest_amount, closing_balance = process_installment_row(
                row, 
                opening_balance, 
                monthly_IR, 
                days_in_month, 
                previous_row_was_crr, 
                crr_info
            )
            
            # Calculate one-day interest for the opening balance
            one_day_interest = opening_balance * monthly_IR / days_in_month
            
            # Calculate the difference between calculated and provided closing balance
            closing_balance_difference = closing_balance - row['Closing Balance']
            
            # Adjust the final closing balance if the difference is less than one-day interest
            if closing_balance_difference < one_day_interest:
                closing_balance -= closing_balance_difference
            
            previous_row_was_crr = False  # Reset after processing a non-CRR row
            rows.append({
                'Installment Date': row['Installment Date'],
                'Transaction Type': row['Transaction Type'],
                'Opening Balance': round(opening_balance, 0),
                'Principal Amount': principal_amount,
                'Interest Amount': interest_amount,
                'Installment Amount': row['Installment Amount'],
                'Closing Balance': closing_balance
            })

    return pd.DataFrame(rows)


In [2]:
from utils.getinfo import IR_from_repayment_schedule
from utils.basic import * 
def repayment_schedule_difference(actual_df,calculated_df):
    difference = abs(actual_df['Closing Balance'] - calculated_df['Closing Balance'])
    monthly_IR,_ = IR_from_repayment_schedule(actual_df)
    
    for i, row in actual_df.iterrows():
        opening_balance = actual_df.at[i, 'Opening Balance']
        day_interest=opening_balance*monthly_IR*(1/30)
        if difference[i] < abs(day_interest):
                difference[i] = 0
        else:
                difference[i] = difference[i]-day_interest
    
    
    difference_sum = np.sum(difference)
    
    return difference, difference_sum

In [3]:
actual_df=pd.read_excel('D:\\test1.xlsx')

In [4]:
from utils.utils import create_repayment_schedule_with_crr
calculated_df=create_repayment_schedule_with_crrr(actual_df)

In [5]:
calculated_df

,Installment Date,Transaction Type,Opening Balance,Principal Amount,Interest Amount,Installment Amount,Closing Balance
0,2023-11-10,Disbursement,0.0,-4500000.00,0.00,-4500000,4500000.00
1,2023-12-10,Installment,4500000.0,23785.00,32250.00,56035,4476215.00
2,2024-01-10,Installment,4476215.0,23955.46,32079.54,56035,4452259.54
3,2024-01-18,Capital Repayment Receivable,4452260.0,3000000.00,0.00,0,1452259.54
4,2024-02-10,Installment,1452260.0,39893.81,16141.19,56035,1412151.87
5,2024-03-10,Installment,1412152.0,45914.58,10120.42,56035,1366237.29
6,2024-04-10,Installment,1366237.0,46243.63,9791.37,56035,1319993.66
7,2024-05-10,Installment,1319994.0,46575.04,9459.96,56035,1273418.61
8,2024-06-10,Installment,1273419.0,46908.83,9126.17,56035,1226509.78
9,2024-07-10,Installment,1226510.0,47245.01,8789.99,56035,1179264.77


In [6]:
df=abs(actual_df['Closing Balance']-calculated_df['Closing Balance'])
df

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
Name: Closing Balance, dtype: float64

In [13]:
new_df=pd.DataFrame({'Difference':df,'daily':daily_interest})
new_df

,Difference,daily
0,0.00,0.000000
1,0.00,1075.000050
2,0.00,1069.318078
3,0.00,1063.595384
4,213.86,346.928684
5,215.39,337.347407
6,216.93,326.378923
7,218.49,315.331833
8,220.05,304.205571
9,221.62,292.999572


In [7]:
yoh=repayment_schedule_difference(actual_df,calculated_df)

In [8]:
yoh

(0       0.000000
 1       0.000000
 2       0.000000
 3       0.000000
 4       0.000000
 5       0.000000
 6       0.000000
 7       0.000000
 8       0.000000
 9       0.000000
 10      0.000000
 11      0.000000
 12      0.000000
 13      0.000000
 14      0.000000
 15      7.263217
 16     20.703617
 17     34.238443
 18     47.868299
 19     61.593796
 20     75.425547
 21     89.344172
 22    103.370290
 23    117.504532
 24    131.737528
 25    146.059914
 26    160.492328
 27    175.035419
 28    188.669838
 29    203.406233
 30    218.255270
 31    233.197609
 32    248.253918
 Name: Closing Balance, dtype: float64,
 2262.419973041968)